In [1]:


import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import urllib

import pandas as pd 
import numpy as np

import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

os.chdir("/Users/janechoi/Desktop/2020-CO-DEEP/preprocess crawl_12_05")

print(os.getcwd())

data00 = pd.read_csv('book_100000.csv')
data0 = pd.read_csv('book_110000.csv')
data01 = pd.read_csv('book_120000.csv')
data1 = pd.read_csv('book_130001_135000.csv')
data2 = pd.read_csv('book_135000_140000.csv')
data3 = pd.read_csv('book_140001_145000.csv')
data4 = pd.read_csv('book_145001_150000.csv')
data5 = pd.read_csv('book_150001_160000.csv')
data6 = pd.read_csv('book_160001_196940.csv')

data3.head()

/Users/janechoi/Desktop/2020-CO-DEEP/preprocess crawl_12_05


,book_id,img,page,pub,pub_year,subject,writer
0,EM0999845,Y,287,마음의 숲,2016,나는 나로 살기로 했다 :냉담한 현실에서 어른살이를 위한 to do list,김수현 글·그림
1,EM0992601,Y,174,꾸리에북스,2016,모든 사람은 혼자다 :결혼한 독신녀 보부아르의 장편 에세이,시몬 드 보부아르 지음 ;박정자 옮김
2,EM0828253,Y,216,그린비,2009,"굿바이 다윈? :신다윈주의, 비판적으로 읽기",이케다 기요히코(池田淸彦) 지음 ;박성관 옮김
3,EE0443423,Y,300,이음,2011,사회생물학 대논쟁,"김동광, 김세균, 최재천 엮음"
4,EM0707567,Y,227,문음사,2004.,(국가가 필요없는) 교육,제임스 툴리 지음 ;손준종 옮김.


### 1. 여기 데이터 고쳐야 됩니다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [2]:
#CONCAT DATA AND SELECT NEEDED 


def concat_data():
    data = pd.concat([data00,data0,data01,data1,data2,data3,data4,data5,data6],axis=0)

    data = data[['book_id','img','pub_year','subject','page']]

    return data


data =concat_data()
data.head(20)
data.isnull().sum()

book_id         0
img             0
pub_year     5873
subject      2381
page        15662
dtype: int64

In [3]:

data.head(60)

,book_id,img,pub_year,subject,page
0,EE0295062,Y,2003.,(쇼펜하우어의) 토론의 법칙 :논쟁과 토론에서 이기는 38가지 기술,136
1,EM0970277,Y,2015,그 사람이 나를 도와주는 진짜 이유 :전문가가 읽어주는 아들러 실전심리학,186
2,EM0970083,Y,2015,"아들러 심리학 입문 :오늘을 살아가는 무기, 용기의 심리학",NaN
3,EM0557577,Y,2001,마이너리그 :은희경 장편소설,250
4,EE0426996,Y,2011.,조경디자인 그래픽랭귀지,183
5,EE0429658,Y,2011.,조경기초디자인,207
6,EE0442934,Y,2011,어떤 작위의 세계 :정영문 장편소설,294
7,EE0447671,Y,2013,땅심 살리는 퇴비 만들기 :석종욱이 들려주는 내 땅 살리는 퇴비제조법,259
8,EM0914862,Y,2013,IFRS 중급회계,NaN
9,EE0370688,Y,2007.,유기농업 :참먹거리 생산의 이론과 기술,330


In [4]:
#CLEAN PUB YEAR 

def clean_pub_year(x):
    
    try: 
        float(x)
        return float(x)
    
    except:
#         print(x)
        r = list(x)        
        s=''
        for i in r:
            if i.isdigit():
                s+=str(i)
            else:
                pass
            
        #for years which have duration 
#         print(s)
        s = s[:4]
    
        #for misinfo 
        if s == '':
            return np.nan 
        else:
            return float(s)
                
        

data['pub_year'] = data['pub_year'].apply(lambda x: clean_pub_year(x))
        
# clean_pub_year('ad')


In [5]:
#CLEAN PAGE NUMS 

#'cm 포함 == 페이지 수 틀림 '
# data.loc[data['book_id'] == 'EE0316889','page'] =736
# data.loc[data['book_id'] == 'EE0316889','page'] 

def clean_page(x):
    try: 
        float(x)
        return float(x)
    
    except: 
        if 'cm' in x: 
#             print('cm in x', str(x))
            return np.nan
        else:
            if ',' in x:
#                 print(x)
                try:
                    return float(x.split(',')[-1])
                except:
                    return np.nan
                
            elif ';' in x:
#                 print(x)
                try:
                    return float(x.split(';')[-1])
                except:
                    return np.nan
                
        
# clean_page('xi,529')


data['page'] = data['page'].apply(lambda x: clean_page(x))

# data.tail()

In [6]:
def clean_column(data):
    data['page'] = data['page'].apply(lambda x: clean_page(x))
    data['pub_year'] = data['pub_year'].apply(lambda x: clean_pub_year(x))
    return data 

data = clean_column(data)
    
data.head(20)

,book_id,img,pub_year,subject,page
0,EE0295062,Y,2003.0,(쇼펜하우어의) 토론의 법칙 :논쟁과 토론에서 이기는 38가지 기술,136.0
1,EM0970277,Y,2015.0,그 사람이 나를 도와주는 진짜 이유 :전문가가 읽어주는 아들러 실전심리학,186.0
2,EM0970083,Y,2015.0,"아들러 심리학 입문 :오늘을 살아가는 무기, 용기의 심리학",NaN
3,EM0557577,Y,2001.0,마이너리그 :은희경 장편소설,250.0
4,EE0426996,Y,2011.0,조경디자인 그래픽랭귀지,183.0
5,EE0429658,Y,2011.0,조경기초디자인,207.0
6,EE0442934,Y,2011.0,어떤 작위의 세계 :정영문 장편소설,294.0
7,EE0447671,Y,2013.0,땅심 살리는 퇴비 만들기 :석종욱이 들려주는 내 땅 살리는 퇴비제조법,259.0
8,EM0914862,Y,2013.0,IFRS 중급회계,NaN
9,EE0370688,Y,2007.0,유기농업 :참먹거리 생산의 이론과 기술,330.0


In [7]:
#SELECT NEW INFO WE NEED TO CRAWL 

info_need = data[(data['img']=='N') | (data['pub_year'].isnull())
                 |(data['subject'].isnull()) | (data['page'].isnull())]

books = info_need['book_id'].unique()
# books[:10]
len(books) #30276

30276

In [8]:
def select_new(data):
    info_need = data[(data['img']=='N') | (data['pub_year'].isnull())
                 |(data['subject'].isnull()) | (data['page'].isnull())]
    
    new_data = data[(data['img']!='N') & (data['pub_year'].notnull())
                 &(data['subject'].notnull()) & (data['page'].notnull())]
    
    books = info_need['book_id'].unique()
    return info_need, new_data, books 



In [9]:
#delete the books according to id

### 위에, 자기 데이터에 맞게 고치고, 그외엔 함수만 실행해도 되고, 이거 실행 시키묜 댄다!!!!!!!!!!!!!!!!!!!! 

In [10]:
##PREPROCESS TOTAL 
data =concat_data()
data = clean_column(data)
new, newdata, books = select_new(data)
len(books) #30276
# newdata.isnull().sum()
newdata.shape #(45999, 5)
new.head(10)
# newdata.to_csv('book_100000_196940.csv', index=0)
data.shape #(96937, 5)

(96937, 5)

In [11]:
new[545:]

,book_id,img,pub_year,subject,page
1431,EM0945159,N,NaN,NaN,NaN
1432,EE0465682,Y,2013.0,나의 문화유산답사기 :일본편,NaN
1433,EE0274908,N,1996.0,일반역학,345.0
1443,EM0964094,Y,2015.0,근대문화사 :흑사병에서 1차 세계대전에 이르기까지 유럽 영혼이 직면한 위기,NaN
1445,EM0883257,Y,2011.0,헌법의 풍경 :잃어버린 헌법을 위한 변론,NaN
...,...,...,...,...,...
36923,EM1012580,Y,NaN,Arirang korean basics,NaN
36924,EM0666036,Y,2017.0,(회사에서 맨날 쓰는) 비즈니스 영어패턴 500+,NaN
36926,EM1015146,N,NaN,South Korea's demographic dividend :echoes of ...,NaN
36927,EM0688769,Y,2017.0,형법요론 =Criminal law,NaN


In [28]:

bookdb= pd.read_csv('new_books_jinhang.csv')
bookdb.shape
# bookdb.tail(38)
bookdb.head(14607)
bookdb_new = bookdb[:14607]
bookdb_new.tail()

,book_id,img,page,pub_year,subject
14602,WM0264999,N,"vi, 400",c2010.,J. M. Coetzee and ethics :philosophical perspe...
14603,EM0886832,Y,484,2008.,일상생활의 사회학적 이해 =Sociological understanding of e...
14604,EE0406875,N,2 冊(1450,1998.,신경과학
14605,WW0139491,N,"xii, 673",c2008.,Adult neurogenesis
14606,EM0662734,Y,563,2004.,윤동주 평전 :나의 청춘은 다하지 않았다


In [79]:
bookdb_new= pd.read_csv('new_books_END.csv')
bookdb_new.shape
bookdb_new.tail()

,book_id,img,page,pub_year,subject
17790,EM0977603,Y,2205,2016,부가가치세 실무
17791,EM1000774,Y,403,2017,페미니즘 위대한 역사
17792,EE0151948,N,252,1996.,게임이론과 남북한 관계 :갈등과 협상 및 예측
17793,WM0177861,Y,1 v. (various pagings) :ill. (some col.) ;26 cm.,c2002.,Investments :a global perspective
17794,EM0948020,Y,책 :삽화 ;27 cm,2011-,訓局謄錄


In [80]:
#NEW CRAWLING
import time
# bookdb_new = pd.DataFrame()

driver = webdriver.Chrome('/Users/janechoi/Desktop/2020-CO-DEEP/crawling-12/chromedriver') 

driver.implicitly_wait(3)
    
for i in tqdm(range(17795,20000)):
    try:
        time.sleep(1)
        book_id = books[i]
    
        url = "https://lib.skku.edu/#/search/total/si?all=1%7Ck%7Ca%7C{}".format(book_id)
        driver.get(url)
    
    
        result = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/ik-total-search[1]/div/div[2]/div/div[2]/ul/li[1]/a[2]')
        result.click()
    
        driver.switch_to.window(driver.window_handles[1])
    
    #open the detailed info 
        try:
            driver.find_element_by_css_selector('#btn-biblio-more-open').click()
        except: 
            pass

    
    #image retreival 
        book_cover = driver.find_elements_by_css_selector('div.ikc-bookcover')
        img_url = re.findall('(?<=src=").+?(?=")', book_cover[0].get_attribute('outerHTML'))[0]
    
        try :
            img_name = 'cover/' + book_id + '.jpg'
            urllib.request.urlretrieve(img_url,img_name)
            img = 'Y'
        
        except:
            img = 'N'
    
    
    
    ##book title 
        book_title = driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child(1) > span').text

        title = book_title.split('/')[0]
    
        for i in range(3,9):
            try:
                text= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child('+str(i)+') > label').text
       
        ##book pub year 
                if text == '발행사항':
                    book_pub= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child('+str(i)+') > span > span').text
                
                    pubyear= book_pub.split(',')[-1]
#                 print(pubyear)
        
        ## book page 
                elif text == '형태사항':
                    book_page= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child('+str(i)+') > span > span').text
                
                    page= book_page.split(' p.')[0]
#                 print(page)
    
                else: 
                    pass
            except:
                break
    
        if pubyear == None: 
            pubyear= np.nan
    
        if page == None:
            page = np.nan
    
        bookdb_new = bookdb_new.append({"book_id" : book_id, "subject" : title, "pub_year" : pubyear, 
    
                                        "page" : page, 'img' : img}, ignore_index=True)
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
    
    except:
        bookdb_new =bookdb_new.append({"book_id" : book_id, 'subject' : 'notfound'
                                       ,'pub_year' : 'notfound', 'page': 'notfound',
                                      'img':'notfound'}, ignore_index=True)
        
    
    
    bookdb_new.to_csv('new_books_jinhang.csv', index = False)
    


100%|██████████| 2205/2205 [2:02:40<00:00,  3.34s/it]  


In [81]:
# bookdb_new.tail(10538)
bookdb_new.shape
bookdb_new.tail(2205)
bookdb_new.head(17795)

# bookdb_new=bookdb_new[:17795]
bookdb_new.tail()

,book_id,img,page,pub_year,subject
19995,EM0948866,Y,302,2013,괴롭히는 아이 당하는 아이 구경하는 아이 :학교폭력의 이해와 예방을 위한 실천방법
19996,EM0892506,Y,6책 :채색삽도 ;30 cm. +전자광디스크(CD-ROM) 각 1매.,2007-2010.,(연세 한국어)활용연습
19997,EM0892486,Y,6책 :채색삽도 ;30 cm. +전자광디스크(CD-ROM) 각 1매.,2007-2010.,(연세 한국어)읽기 =Yonsei Korean reading
19998,EE0046319,N,225,1991.,하느님의 기쁨 사람의 희망
19999,EM0734222,Y,"viii, 322",2002 2005 개정 3쇄.,통계학 입문


In [ ]:
# new[new['subject'].isnull()]
new.head(10)

In [82]:
bookdb_new.to_csv('new_books_END.csv', index = False)

In [85]:
bookdb_new.head(40)

,book_id,img,page,pub_year,subject
0,EM0970083,Y,240,2015,"아들러 심리학 입문 :오늘을 살아가는 무기, 용기의 심리학"
1,EM0914862,Y,"xxvi, 1200",2013,IFRS 중급회계
2,EM0912722,Y,303,2013,(IFRS) 중급회계 :연습문제해답
3,EE0265015,N,"vi, 393",2000.,작물재배학원론
4,EE0428941,N,2冊.(1182,2011.,(제4판)사료자원핸드북
5,EE0391459,N,"xiii, 158",2007.,(한국사양표준)한우 =Korean feeding standard for Hanwoo...
6,EE0428940,N,2冊.(1182,2011.,(제4판)사료자원핸드북
7,EE0424603,N,책 :삽화 ;26 cm,1981-,농촌지도사업보고서
8,EE0464349,N,"책 :삽화, 표 ;26 cm",1997-,주요 농산물 유통실태
9,EM0561638,Y,"ii, 221",2000.,수학 100의 정리 :피타고라스 정리부터 현대수학까지


In [ ]:
book  = pd.read_csv('book_100000_196940.csv')
# book.head(20)